# Spheres

Fivethirtyeight's Riddler asks "[what is the minimum number of spheres of sequential size (1,2,3, etc.) that can be divided into three equal-sized groups](https://fivethirtyeight.com/features/can-you-flip-the-magic-coin/)? By "equal size", the problem refers to weight, but since all the spheres in this case are made of the same material (gold, as it happens), this is equivalent to finding groups of equal volume. We can further simplify the problem: the volume of a sphere of radius $r$ is $\frac{4}{3}\pi r^3$; we can pull $\frac{4}{3}\pi$ out of each volume to make this equivalent to finding the smallest $n$ such that the first $n$ cubes can be partitioned into three groups of equal sum.

I will use Julia to work this problem, in an attempt to learn more of the language. We start with a function for the sum of cubes:

In [1]:
function sumofcubes(n)
    return (n * (n+1) / 2)^2
end

sumofcubes (generic function with 1 method)

The first step in finding a partitionable set is to find **one** partition, so let's create an algorithm to do that.

In [2]:
# Algorithm:
# Calculate one third of the sum of cubes
# Generate a sorted list of cubes
# For each element in the list of cubes
#     put this element in a partition
#     decrement the total by that amount
#     try to find a subset of the remaining elements adding to the lower total

function findpart(list, total)
    for idx in 1:length(list)
        elt = list[idx]
        if elt > total
            break # everything left is also too big
        end
        if elt == total # jackpot
            return [elt]
        end
        rest = findpart(list[idx+1:end], total - elt)
        if !isempty(rest)
            return append!([elt],rest)
        end
    end
    return []
end

findpart (generic function with 1 method)

We can eliminate certain values of $n$ as possibilities immediately. For one, if $n = 1 \left(\mod 3\right)$, then the sum of cubes won't be divisible by 3. Also, the sum of the cubes has to be larger than three times the largest cube; the smallest $n$ where this occurs is 10, but since this is already not a possibility, we can start with $n = 11$. 

In [3]:
n = 10 # I know I just said start at 11, but...
cubes = map(x -> x^3, 1:n)
result = []
while isempty(result) # loop until we find a solution
    n += 1 # ... adding one to n up front allows a quick continue
    append!(cubes, n^3)
    if n % 3 == 1
        continue
    end
    total = sumofcubes(n)/3
    firstpart = findpart(cubes, total)
    if !isempty(firstpart)
        secondpart = findpart(filter(x -> !(x in firstpart), cubes), total)
        if !isempty(secondpart) # found two, third group has to work
            result = [n, 
                firstpart, 
                secondpart, 
                filter(x -> !(x in vcat(firstpart, secondpart)), cubes)]
            break
        end
    end
end
result 

4-element Array{Any,1}:
 35                                                                                                               
   [1, 8, 27, 64, 125, 216, 343, 512, 729, 1000  …  1728, 2197, 2744, 3375, 4096, 4913, 9261, 24389, 35937, 39304]
   [5832, 12167, 19683, 21952, 29791, 42875]                                                                      
   [6859, 8000, 10648, 13824, 15625, 17576, 27000, 32768]                                                         

Huzzah!
Not the most efficient or elegant algorithm ever, but we've got a solution.

## Extra Credit

Now, we are asked what the minimum number of spheres are that can be partitioned into 2, 4, 5, 6, or k sets. We've got the tools in place, we just need to change the loop a bit:

1. We need to pass the number of needed sets, instead of assuming it's 3.
2. We should also just check that the total needed for each partition is an integer.
3. A nested loop is no longer feasible, so we have to create a recursive function.

In [4]:
function partitioncubes(k)
    if k == 1
        return [[1], [1]]
    end
    n = 4k - 2 # if n > 1, this is the lowest n for which k*n^3 is less than the sum of the fist n cubes
    result = []
    while true
        total = sumofcubes(n)/k
        if total % 1 == 0 # no fractional part
            cubes = map(x -> x^3, 1:n) # we're going to filter this list in place, so recreate it every time
            for j = 1:k
                part = findpart(cubes, total)
                if isempty(part)
                    empty!(result) # reset the result
                    break # for loop
                else
                    push!(result, part)
                    filter!(x -> !(x in part), cubes)
                end # ifelse
            end # for loop
            if !isempty(result)
                break # while loop
            end
        end # if total...
        n += 1
    end # while loop
    return append!([[n]], result)
end

partitioncubes (generic function with 1 method)

Let's see how this goes:

In [5]:
partitioncubes(2)

3-element Array{Array{Int64,1},1}:
 [12]                           
 [1, 8, 64, 512, 729, 1728]     
 [27, 125, 216, 343, 1000, 1331]

In [6]:
partitioncubes(3) # better get 35

4-element Array{Array{Int64,1},1}:
 [35]                                                                                                           
 [1, 8, 27, 64, 125, 216, 343, 512, 729, 1000  …  1728, 2197, 2744, 3375, 4096, 4913, 9261, 24389, 35937, 39304]
 [5832, 12167, 19683, 21952, 29791, 42875]                                                                      
 [6859, 8000, 10648, 13824, 15625, 17576, 27000, 32768]                                                         

In [7]:
partitioncubes(4)

5-element Array{Array{Int64,1},1}:
 [99]                                                                                                                            
 [1, 8, 27, 64, 125, 216, 343, 512, 729, 1000  …  185193, 195112, 205379, 216000, 226981, 238328, 262144, 357911, 778688, 912673]
 [250047, 274625, 287496, 300763, 314432, 328509, 373248, 389017, 438976, 512000, 551368, 571787, 729000, 804357]                
 [343000, 456533, 493039, 531441, 614125, 636056, 658503, 704969, 830584, 857375]                                                
 [405224, 421875, 474552, 592704, 681472, 753571, 884736, 941192, 970299]                                                        

In [8]:
partitioncubes(5)

6-element Array{Array{Int64,1},1}:
 [174]                                                                                                                                                                                  
 [1, 8, 27, 64, 125, 216, 343, 512, 729, 1000  …  1157625, 1191016, 1225043, 1259712, 1404928, 1685159, 1906624, 2000376, 2146689, 2571353]                                             
 [1295029, 1331000, 1367631, 1442897, 1481544, 1520875, 1560896, 1601613, 1643032, 1728000  …  1860867, 1953125, 2048383, 2097152, 2197000, 2352637, 2744000, 3869893, 4019679, 4657463]
 [2248091, 2299968, 2406104, 2460375, 2515456, 2628072, 2803221, 2985984, 3241792, 4096000, 4173281, 4492125, 4741632, 5268024]                                                         
 [2685619, 2863288, 3307949, 3375000, 3442951, 3511808, 3944312, 4330747, 4410944, 4574296, 4913000, 5000211]                                                                           
 [2924207, 3048625, 3112136, 3176523, 35

In [9]:
partitioncubes(6)

7-element Array{Array{Int64,1},1}:
 [215]                                                                                                                                                                                  
 [1, 8, 27, 64, 125, 216, 343, 512, 729, 1000  …  2000376, 2048383, 2097152, 2146689, 2197000, 2803221, 3048625, 3511808, 3581577, 4410944]                                             
 [2248091, 2299968, 2352637, 2406104, 2460375, 2515456, 2571353, 2628072, 2685619, 2744000  …  3375000, 3442951, 3652264, 3723875, 4019679, 4096000, 4251528, 4492125, 5639752, 6644672]
 [3796416, 3869893, 3944312, 4173281, 4330747, 4574296, 4657463, 4741632, 4826809, 4913000, 5088448, 5268024, 6028568, 6128487, 6229504, 7762392, 9528128]                              
 [5000211, 5177717, 5359375, 5451776, 5735339, 7189057, 7414875, 7529536, 7645373, 7880599, 8120601, 8615125, 8741816]                                                                  
 [5545233, 5832000, 6539203, 6751269, 68

Is this enough for a pattern? The results themselves aren't very helpful. But notice that the first partition each time contains a long set of consecutive cubes. Perhaps this will give us insight?

In [10]:
function consecutivecubes(k)
    part = partitioncubes(k)[2] # partition contaiing the smallest cubes
    current = 1
    while current^3 in part
        current += 1
    end
    return current - 1 # this is a silly way to do this, isn't it?
end

consecutivecubes (generic function with 1 method)

In [11]:
consecutivecubes(2) # should be 2

2

In [12]:
consecutivecubes(3) # should be 17

17

In [13]:
println(consecutivecubes(4))
println(consecutivecubes(5))
println(consecutivecubes(6))

62
108
130


In [14]:
a = partitioncubes(7)

8-element Array{Array{Int64,1},1}:
 [272]                                                                                                                                                                                      
 [1, 8, 27, 64, 125, 216, 343, 512, 729, 1000  …  3723875, 3796416, 3869893, 3944312, 4019679, 4096000, 5832000, 7077888, 7762392, 10360232]                                                
 [4173281, 4251528, 4330747, 4410944, 4492125, 4574296, 4657463, 4741632, 4826809, 4913000  …  6229504, 6331625, 6434856, 6539203, 6644672, 6751269, 8615125, 10941048, 12008989, 14706125] 
 [6859000, 6967871, 7189057, 7301384, 7414875, 7529536, 7645373, 7880599, 8000000, 8120601  …  8489664, 8741816, 8869743, 8998912, 9800344, 9938375, 10077696, 10793861, 12326391, 17373979]
 [9129329, 9261000, 9393931, 9528128, 9663597, 10218313, 10503459, 10648000, 11089567, 11239424, 12487168, 13144256, 13312053, 13481272, 13651919, 14172488, 16003008]                      
 [11390625, 11543176

In [15]:
maximum(filter(x->x^3 in a[2], 1:length(a[2]))) # perhaps a more "Juliaic" way to do this?

160

This seems unhelpful. Perhaps there's an asymptotic result for k. Perhaps I'll see it someday.